### Preparing the tools

We 're going to use pandas, Matplotlib and NumPy for data analysis and manipulation.

In [ ]:
# Import all tools we need

# Regular EDA (exploratory data analysis) and plotting libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# we want our plots to appear inside the notebook
%matplotlib inline 

# Models from Scikit-Learn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Model Evaluations
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import plot_roc_curve

## Load data

In [ ]:
df = pd.read_csv('/kaggle/input/heartdisease/data/heart-disease.csv', delimiter=',', nrows=None)
df.shape 

## Data Exploration(EDA )

The goal here is to find out more about the data and become a subject matter expert ondata you're working with. 

1. What questions are you trying to solve?
2. What kind of data do we have and how do we treat different types?
3. What's missing from the data and how do you deal with it?
4. Where are the outliers and why should you care about it?
5. How can you add, change or remove features to get more out of your data? 

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
# Let's find out how many of each classes there
df.target.value_counts()

In [ ]:
df.target.value_counts().plot(kind="bar", color=["salmon", "lightblue"]);

In [ ]:
df.info()

In [ ]:
# Are there any missing values?
df.isna().sum()

In [ ]:
df.describe()

### Heart Disease Frequency according to Sex

In [ ]:
df.sex.value_counts()

In [ ]:
# Compare target column with sex column  
pd.crosstab(df.target, df.sex)

In [ ]:
# Create a plot of crosstab
pd.crosstab(df.target, df.sex).plot(kind="bar", 
                                    color=["salmon", "lightblue"],
                                    figsize=(10, 6));

plt.title("Heart Disease Frequency for Sex")
plt.xlabel("0 = No Disease, 1 = Disease")
plt.ylabel("Amount")
plt.legend(["Female", "Male"]);
plt.xticks(rotation=0)

In [ ]:
df.head()

### Age vs Max Heart Rate for Heart Disease

In [ ]:
# Creating another figure
plt.figure(figsize=(10, 6))

# Scatter with positive examples
plt.scatter(df.age[df.target == 1],
            df.thalach[df.target == 1],
            color = "salmon")

# Scatter with negetive examples
plt.scatter(df.age[df.target == 0],
            df.thalach[df.target == 0],
            color = "lightblue")

In [ ]:
df.age[df.target == 1]

In [ ]:
# Check the distribution of age column with histogram
df.age.plot.hist();

## Heart Disease Frequency per chest pain type
3. cp - chest pain type

* Typical angina: chest pain related decrease blood supply to the heart
* Atypical angina: chest pain not related to heart
* Non-anginal pain: typically esophageal spasms (non heart related)
* Asymptomatic: chest pain not showing signs of disease

In [ ]:
pd.crosstab(df.cp, df.target)

In [ ]:
# Make the crosstab more visual 
pd.crosstab(df.cp, df.target).plot(kind="bar",
                                   figsize=(10, 6),
                                   color = ["red","blue"])

# Add some communication
plt.title('Heart Disease Freq per chest pain type')
plt.xlabel('Chest pain type')
plt.ylabel("Amount")
plt.legend(["No Disease", "Disease"])
plt.xticks(rotation = 0);

In [ ]:
df.head()

In [ ]:
# Make a correlation matrix
df.corr()

In [ ]:
# Let's make our correlation matrix more prettier
corrMatrix = df.corr()
fig, ax = plt.subplots(figsize=(15, 10))
ax = sns.heatmap(corrMatrix,
                 annot = True,
                 linewidths = 0.5,
                 fmt = ".2f",
                 cmap = "YlGnBu")


## 5. Modelling

In [ ]:
df.head()

In [ ]:
# Split data to X & y
X = df.drop("target", axis=1)

y  = df.target

In [ ]:
X

In [ ]:
y

In [ ]:
# Split data to train & test sets
np.random.seed(44)

# Split into train & test set
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2)

In [ ]:
X_train

In [ ]:
y_train, len(y_train)

Now we've got our data split into training and test sets, it's time to build a machine learning model

We'll train it (find the patterns) on the training set.

And we'll test it (use the patterns) on the test set.

We're going to try 3 different machine learning models:
1. Logistic Regression
2. K-Nearest Neighbor s Clasifier
3. Random Forest Classifier

In [ ]:
# Put models in a dictionary
models = {"Logistic Regression":LogisticRegression(),
          "KNN":KNeighborsClassifier(),
         "Random Forest":RandomForestClassifier()}

# Create a function to fit and scroe models
def fit_and_score(models, X_train, X_test, y_train, y_test):
    """
    Fits and evaluates given machine learning models.
    models: a dict of different Scikit-Learn machine learning models
    X_train: training data (no labels)
    X_test: testing data(no labels)
    y_train: training labels
    y_test: test labels
    """
    # Set random seed
    np.random.seed(44)
    # Make a dictionary to keep model scores
    model_scores = {}
    # Loop through models
    for name, model in models.items():
        # Fit the model to the data
        model.fit(X_train, y_train)
        # Evaluate the model and append its score to model_scores
        model_scores[name] = model.score(X_test, y_test)
    return model_scores

In [ ]:
model_scores = fit_and_score(models=models,
                             X_train=X_train,
                             X_test=X_test,
                             y_train=y_train,
                             y_test=y_test)

model_scores

## Model Comparison

In [ ]:
models_compare = pd.DataFrame(model_scores, index=["accuracy"])
models_compare.T.plot(kind="bar")

plt.xticks(rotation =0);

Now we've got our baseline model...and we know a model's first predictions aren't always what we should based our next steps off. What should do?

Let's look at the following:
* Hyperparameter tuning
* Feature importance
* Confusion matrix
* Cross-vaidation
* Precision
* Recall
* F1 score
* Classification report
* ROC curve
* Area under the curve (AUC)

### Hyperparameter tuning

In [ ]:
# Let's tune KNN

train_scores = []
test_scores = []

# Create a list of different values for n_neighbors
neighbors = range(1, 21)

# Setup KNN instance
knn = KNeighborsClassifier()

# Loop through different n_neighbors 
for i in neighbors:
    knn.set_params(n_neighbors=i)
    
    # Fit the algorithm
    knn.fit(X_train, y_train)
    
    # Update training scores list
    train_scores.append(knn.score(X_train, y_train))
    
    # Update test scores list
    test_scores.append(knn.score(X_test, y_test))

In [ ]:
train_scores

In [ ]:
test_scores

In [ ]:
plt.plot(neighbors, train_scores, label="Train score")
plt.plot(neighbors, test_scores, label="Test score")
plt.xticks(np.arange(1, 21, 1))
plt.xlabel("No of neighbors")
plt.ylabel("Model score")
plt.legend();

print(f'Maximum KNN score on test data:{max(test_scores)* 100:.2f}%')

## Hyperparameter tuning with RandomizedSearchCV

We're going to tune:
* LogisticRegression()
* RandomForestClassifier()

... using RandomizedSearchCV

In [ ]:
# Create a hyperparameter grid for LogisticRegression
log_reg_grid ={"C":np.logspace(-4, 4, 20),
              "solver": ["liblinear"]}

# Create a hyperparameter grid for RandomForestClassifier
rf_grid ={"n_estimators": np.arange(10, 1000, 50),
          "max_depth":[None, 3, 5, 10],
          "min_samples_split":np.arange(2, 20, 2),
          "min_samples_leaf": np.arange(1, 20, 2)}

Now we've got hyperparameter grids setup for each model, let's tune them RandomizedSearchCV...

In [ ]:
# Tune LogisticRegression

np.random.seed(34)

# Setup random hyperparameter search for LogisticRegression
rs_log_reg = RandomizedSearchCV(LogisticRegression(),
                                param_distributions=log_reg_grid,
                                cv=5,
                                n_iter=20,
                                verbose=True)

# Fit random hyperparameter search model for LogisticRegression
rs_log_reg.fit(X_train, y_train)

In [ ]:
rs_log_reg.best_params_

In [ ]:
rs_log_reg.score(X_test, y_test)

Now we've tuned LogisticRegression(), lets do the same for RandomForestClassifier()...

In [ ]:
# Setup random seed
np.random.seed(44)

# Setup hyperparameter search for RandomForestClassifier
rs_rf = RandomizedSearchCV(RandomForestClassifier(),
                           param_distributions = rf_grid,
                           cv=5,
                           n_iter=2,
                           verbose=True)

# Fit random hyperparamete search for RandomForestClassifier
rs_rf.fit(X_train, y_train)

In [ ]:
# Find the best hyperparameters
rs_rf.best_params_

In [ ]:
# Evaluate the randomized search RandomForestClassifier model
rs_rf.score(X_test, y_test)

In [ ]:
model_scores

## Hyperparameter Tuning with GridSearchCV

Since our LogisticRegression model provides the best scores so far, we'll try and improve them again using GridSearchCV...

In [ ]:
# Different hyperparameter for LogisticRegression model
log_reg_grid = {"C": np.logspace(-4, 4, 30),
               "solver": ['liblinear']}

# Setup grid hyperparameter search for LogisticRegression
gs_log_reg = GridSearchCV(LogisticRegression(),
                          param_grid=log_reg_grid,
                          cv=5,
                          verbose=True)

# Fit grid hyperparameter search model
gs_log_reg.fit(X_train, y_train);

In [ ]:
# Check the best parameters
gs_log_reg.best_params_

In [ ]:
# Evaluate the grid search LogisticRegression model
gs_log_reg.score(X_test, y_test)

## Evaluating our tuned machine learning classifier beyond accuracy

* ROC curve and AUC curve
* Confusion matrix
* Classification report
* Precision 
* Recall
* F1-score 

... and it would be great if cross-validation was used where possible.

In [ ]:
# Make predictions with tuned model
y_preds = gs_log_reg.predict(X_test)

In [ ]:
y_preds

In [ ]:
y_test

In [ ]:
# Plot ROC curve and calculate and AUC metric
plot_roc_curve(gs_log_reg, X_test, y_test);

In [ ]:
# Confusion matrix
confusion_matrix(y_test,y_preds)

In [ ]:
sns.set(font_scale=1.5)

def plot_conf_mat(y_test, y_preds):
    """
    Plots a nice looking confusion matrix using Seaborn's heatmap()
    """
    fig, ax = plt.subplots(figsize=(3,3))
    ax = sns.heatmap(confusion_matrix(y_test, y_preds),
                     annot=True,
                     cbar=True)
    plt.xlabel("True label")
    plt.ylabel("Predicted label")
    
plot_conf_mat(y_test, y_preds)

Now we've got a ROC curve, an AUC metric and a confusion matrix, let's get a classification report as well as cross-validated precision, recall and f1-score.

In [ ]:
print(classification_report(y_test, y_preds))

## Calculate evaluation metrics using cross-validation

We're going to calculate accuracy, precision score, recall and f1-score of our modelusing cross-validation and to do so we'll be using  `cross_val_score()`.

In [ ]:
# Check best hyperparameters
gs_log_reg.best_params_

In [ ]:
# Create a new classifier with best parameters
clf = LogisticRegression(C = 4.893900918477489,
                        solver = "liblinear")

In [ ]:
# Cross-validated accuracy
cv_acc = cross_val_score(clf,
                         X,
                         y,
                         cv=5,
                         scoring = "accuracy")
cv_acc

In [ ]:
cv_acc = np.mean(cv_acc)
cv_acc

In [ ]:
# Cross-validated precision
cv_precision = cross_val_score(clf,
                               X,
                               y,
                               cv=5,
                               scoring = "precision")
cv_precision = np.mean(cv_precision)
cv_precision

In [ ]:
# Cross-validated recall
cv_recall = cross_val_score(clf,
                            X,
                            y,
                            cv=5,
                            scoring = "recall")
cv_recall = np.mean(cv_recall)
cv_recall

In [ ]:
# Cross-validated f1-score
cv_f1 = cross_val_score(clf,
                        X,
                        y,
                        cv=5,
                        scoring = "f1")
cv_f1 = np.mean(cv_f1)
cv_f1

In [ ]:
# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({"Accuracy": cv_acc,
                           "Precision": cv_precision,
                           "Recall":cv_recall,
                           "F1":cv_f1},
                         index= [0])

cv_metrics.T.plot(kind="bar",
                  title= "Cross-validated classification metrics",
                  legend=False)
plt.xticks(rotation=0);

### Feature Importance

Feature importance is another as asking, "which feature contributed most to outcomes of the model and how did they contribute?"

Finding feature importance is different for each machine learning model. One way to find feature importance is to search for '(MODEL NAME) feature importance'.

Lets find feature importance for our LogisticRegression model...

In [ ]:
# Fit an instance of LogisticRegression
clf = LogisticRegression(C = 4.893900918477489,
                        solver='liblinear')
clf.fit(X_train, y_train);

In [ ]:
# Check  coef_
clf.coef_

In [ ]:
X

In [ ]:
X.head()

In [ ]:
# Match coef's of features to columns
feature_dict = dict(zip(df.columns, list(clf.coef_[0])))
feature_dict

In [ ]:
# Visualize feature impoertance
feature_df = pd.DataFrame(feature_dict, index=[0])
feature_df.T.plot(kind="bar",
                  title="Feature Importance",
                  figsize=(6,6),
                 legend = False)
plt.xticks(rotation=90);

In [ ]:
pd.crosstab(df.sex, df.target)

In [ ]:
pd.crosstab(df.slope, df.target)

slope - the slope of the peak exercise ST segment

* 0: Upsloping: better heart rate with excercise (uncommon)
* 1: Flatsloping: minimal change (typical healthy heart)
* 2: Downslopins: signs of unhealthy heart
